In [1]:
import numpy as np
import scipy
import pandas as pd
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import os
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

In [2]:
folder_list=glob('../../data/MixArtifacts/raw/**')
folder_list=[f.replace('/raw/', '/csv/') for f in folder_list]
for i in range(len(folder_list)):
    createDirectory(folder_list[i])
mat_file_list= glob('../../data/MixArtifacts/raw/**/*.mat')
csv_file_list=[f.replace('.mat', '.csv') for f in mat_file_list]
csv_file_list=[f.replace('/raw/', '/csv/') for f in csv_file_list]
for i in tqdm(range(len(mat_file_list))):
    mat_file=scipy.io.loadmat(mat_file_list[i])
    value_array=mat_file['signals'][0][0]
    Type_1=value_array[5][0]
    Type_2=value_array[5][1]
    Type_3=value_array[5][2]
    Type_4=value_array[5][3]
    Type_5=value_array[5][4]
    raw_PPG=value_array[1][0]
    PPG=np.zeros([len(Type_1)],dtype=np.int8)
    prev_index=0
    for j in range(len(raw_PPG)):
        next_index=int(len(Type_1)/len(raw_PPG)*(j+1))
        PPG[prev_index:next_index+1]=raw_PPG[j]
        prev_index=int(len(Type_1)/len(raw_PPG)*(j+1))+1
    data = {'PPG':PPG,'Type_1' : Type_1,'Type_2' : Type_2,'Type_3' : Type_3,'Type_4' : Type_4,'Type_5' : Type_5}
    data=pd.DataFrame(data)
    data.to_csv(csv_file_list[i],mode = 'w', index=False)

  0%|          | 0/2400 [00:00<?, ?it/s]

In [4]:
os.path.basename(os.path.dirname(csv_file_list[0]))

'서맥'

In [7]:
file_count=0
negative_count=0
pulse=3000
waveform_binary_path='../../data/MixArtifacts/b30sec/'
train_binary_df=pd.DataFrame(columns=['FileName','class','AF'])
test_binary_df=pd.DataFrame(columns=['FileName','class','AF'])
val_binary_df=pd.DataFrame(columns=['FileName','class','AF'])
waveform_mean_path='../../data/MixArtifacts/m30sec/'
train_mean_df=pd.DataFrame(columns=['FileName','class','AF'])
test_mean_df=pd.DataFrame(columns=['FileName','class','AF'])
val_mean_df=pd.DataFrame(columns=['FileName','class','AF'])
def csv_save(PPG,Type_1,Type_2,Type_3,Type_4,Type_5,waveform_path,data_set_class,count):
    data = {'PPG Class':PPG,'PPG' : Type_1}
    data_1=pd.DataFrame(data)
    data_1.to_csv(waveform_path+data_set_class+'/Type_1_'+str(count)+'.csv',mode = 'w', index=False)
    data = {'PPG Class':PPG,'PPG' : Type_2}
    data_2=pd.DataFrame(data)
    data_2.to_csv(waveform_path+data_set_class+'/Type_2_'+str(count)+'.csv',mode = 'w', index=False)
    data = {'PPG Class':PPG,'PPG' : Type_3}
    data_3=pd.DataFrame(data)
    data_3.to_csv(waveform_path+data_set_class+'/Type_3_'+str(count)+'.csv',mode = 'w', index=False)
    data = {'PPG Class':PPG,'PPG' : Type_4}
    data_4=pd.DataFrame(data)
    data_4.to_csv(waveform_path+data_set_class+'/Type_4_'+str(count)+'.csv',mode = 'w', index=False)
    data = {'PPG Class':PPG,'PPG' : Type_5}
    data_5=pd.DataFrame(data)
    data_5.to_csv(waveform_path+data_set_class+'/Type_5_'+str(count)+'.csv',mode = 'w', index=False)
    
    
for i in tqdm(range(len(csv_file_list))):
    csv_data=pd.read_csv(csv_file_list[i])
    count=len(csv_data)
    a=np.random.randint(0, 10)
    if a==0:
        data_set_class='test'
    elif a==1:
        data_set_class='val'
    else: 
        data_set_class='train'
    for j in range(count//(pulse-500)-1):
        sec_data=csv_data[j*(pulse-500):j*(pulse-500)+pulse]
        PPG=sec_data['PPG'].to_numpy()
        PPG_mean=PPG.mean()
        Type_1=sec_data['Type_1'].to_numpy()
        Type_2=sec_data['Type_2'].to_numpy()
        Type_3=sec_data['Type_3'].to_numpy()
        Type_4=sec_data['Type_4'].to_numpy()
        Type_5=sec_data['Type_5'].to_numpy()
        file_name_list=[]
        PPG_binary_list=[]
        PPG_mean_list=[]
        class_list=[]
        
        for k in range(1,6):
            file_name_list.append('Type_'+str(k)+'_'+str(file_count)+'.csv')
            PPG_mean_list.append(PPG_mean)
            class_list.append(os.path.basename(os.path.dirname(csv_file_list[i])))
            if PPG_mean>0:
                PPG_binary_list.append(1)
            else: 
                PPG_binary_list.append(0)
                
        if data_set_class=='train':
            csv_save(PPG,Type_1,Type_2,Type_3,Type_4,Type_5,waveform_binary_path,data_set_class,file_count)
            csv_save(PPG,Type_1,Type_2,Type_3,Type_4,Type_5,waveform_mean_path,data_set_class,file_count)
            binary_data={'FileName':file_name_list,'class':class_list,'AF':PPG_binary_list}
            mean_data={'FileName':file_name_list,'class':class_list,'AF':PPG_mean_list}
            PPG_binary_temp=pd.DataFrame(binary_data)
            PPG_mean_temp=pd.DataFrame(mean_data)
            train_binary_df = pd.concat([train_binary_df,PPG_binary_temp])
            train_mean_df = pd.concat([train_mean_df,PPG_mean_temp])
            file_count+=1
            
        if data_set_class=='test':
            csv_save(PPG,Type_1,Type_2,Type_3,Type_4,Type_5,waveform_binary_path,data_set_class,file_count)
            csv_save(PPG,Type_1,Type_2,Type_3,Type_4,Type_5,waveform_mean_path,data_set_class,file_count)
            binary_data={'FileName':file_name_list,'class':class_list,'AF':PPG_binary_list}
            mean_data={'FileName':file_name_list,'class':class_list,'AF':PPG_mean_list}
            PPG_binary_temp=pd.DataFrame(binary_data)
            PPG_mean_temp=pd.DataFrame(mean_data)
            test_binary_df = pd.concat([test_binary_df,PPG_binary_temp])
            test_mean_df = pd.concat([test_mean_df,PPG_mean_temp])
            file_count+=1
        if data_set_class=='val':
            csv_save(PPG,Type_1,Type_2,Type_3,Type_4,Type_5,waveform_binary_path,data_set_class,file_count)
            csv_save(PPG,Type_1,Type_2,Type_3,Type_4,Type_5,waveform_mean_path,data_set_class,file_count)
            binary_data={'FileName':file_name_list,'class':class_list,'AF':PPG_binary_list}
            mean_data={'FileName':file_name_list,'class':class_list,'AF':PPG_mean_list}
            PPG_binary_temp=pd.DataFrame(binary_data)
            PPG_mean_temp=pd.DataFrame(mean_data)
            val_binary_df = pd.concat([val_binary_df,PPG_binary_temp])
            val_mean_df = pd.concat([val_mean_df,PPG_mean_temp])
            file_count+=1
train_binary_df.to_csv(waveform_binary_path+'train.csv',encoding='cp949', index=False)
test_binary_df.to_csv(waveform_binary_path+'test.csv',encoding='cp949', index=False)
val_binary_df.to_csv(waveform_binary_path+'val.csv',encoding='cp949', index=False)
train_mean_df.to_csv(waveform_mean_path+'train.csv',encoding='cp949', index=False)
test_mean_df.to_csv(waveform_mean_path+'test.csv',encoding='cp949', index=False)
val_mean_df.to_csv(waveform_mean_path+'val.csv',encoding='cp949', index=False)

  0%|          | 0/2400 [00:00<?, ?it/s]

/tmp/ipykernel_166369/931564594.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  train_mean_df = pd.concat([train_mean_df,PPG_mean_temp])
/tmp/ipykernel_166369/931564594.py:92: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  val_mean_df = pd.concat([val_mean_df,PPG_mean_temp])
/tmp/ipykernel_166369/931564594.py:82: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when dete